**Общая информация**

**Срок сдачи:** 13 марта 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 13 марта, -4 балла после 06:00 20 марта, -6 баллов после 06:00 27 марта

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе.

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn 4 балла для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 3 (2 балла)
Выполнить требования pep8

###### Задание 5 (2 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)


In [22]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier



In [23]:
from collections import Counter
from scipy.spatial.distance import cdist
from scipy.stats import mode
import operator

class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin, ClassifierMixin):
    
    def __init__(self, n_neighbors):
        self.k = n_neighbors
        self.X = np.zeros(0)
        self.y = np.zeros(0)
        self.N_classes = 0
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.N_classes = len(Counter(y.tolist()))
        
        
    def predict(self, X_test):
        dst = cdist(X_test, self.X)
        y_metrics = self.y[np.argsort(dst)[:, :self.k]]
        return mode(y_metrics, axis = 1).mode.reshape((-1,))
        
            
                
                
            
    
    def predict_proba(self, X_test):
        dst = cdist(X_test, self.X)
        ans = np.zeros((X_test.shape[0], self.N_classes))
        y_metrics = self.y[np.argsort(dst)[:, :self.k]]
        for i in range(y_metrics.shape[0]):
            for j in range(y_metrics.shape[1]):
                ans[i][y_metrics[i][j].astype(int)] += 1 
        ans /= self.k
        return ans
    
    def score(self, X, y):
        print self.predict(X),  y
        return 1 - (0. + len(np.nonzero(self.predict(X) - y))) / y.shape[0]
        

**IRIS**

In [24]:
iris = datasets.load_iris()

In [25]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)


In [26]:
clf = KNeighborsClassifier(n_neighbors=10) 
my_clf = MyKNeighborsClassifier(n_neighbors=10)
%time clf.fit(X_train, y_train)

CPU times: user 1.86 ms, sys: 5 µs, total: 1.86 ms
Wall time: 1.24 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [27]:
%time my_clf.fit(X_train, y_train)

CPU times: user 91 µs, sys: 0 ns, total: 91 µs
Wall time: 73.9 µs


In [28]:
%time clf.predict(X_test)

CPU times: user 769 µs, sys: 3 µs, total: 772 µs
Wall time: 589 µs


array([1, 1, 2, 2, 2, 1, 2, 0, 1, 2, 0, 0, 0, 0, 1])

In [29]:
%time my_clf.predict(X_test)

CPU times: user 468 µs, sys: 0 ns, total: 468 µs
Wall time: 255 µs


array([1, 1, 2, 2, 2, 1, 2, 0, 1, 2, 0, 0, 0, 0, 1])

In [30]:
%time clf.predict_proba(X_test)

CPU times: user 819 µs, sys: 0 ns, total: 819 µs
Wall time: 522 µs


array([[0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 0.4, 0.6],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0.7, 0.3]])

In [31]:
%time my_clf.predict_proba(X_test)

CPU times: user 3.13 ms, sys: 0 ns, total: 3.13 ms
Wall time: 1.9 ms


array([[0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 0.4, 0.6],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0.7, 0.3]])

In [32]:
clf.score(X_test, y_test)

1.0

In [33]:
my_clf.score(X_test, y_test)

[1 1 2 2 2 1 2 0 1 2 0 0 0 0 1] [1 1 2 2 2 1 2 0 1 2 0 0 0 0 1]


0.9333333333333333

In [13]:
# Задание 3

**MNIST**

In [34]:
mnist = fetch_mldata('MNIST original')

In [35]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.1, stratify=mnist.target)

In [36]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [37]:
%time clf.fit(X_train, y_train)

CPU times: user 12.4 s, sys: 84 ms, total: 12.5 s
Wall time: 12.5 s


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [38]:
%time my_clf.fit(X_train, y_train)

CPU times: user 17.1 ms, sys: 3.99 ms, total: 21.1 ms
Wall time: 15.4 ms


In [39]:
%time clf.predict(X_test)

CPU times: user 7min 41s, sys: 16 ms, total: 7min 41s
Wall time: 7min 41s


array([5., 0., 8., ..., 2., 2., 7.])

In [40]:
%time my_clf.predict(X_test)

CPU times: user 3min 13s, sys: 1.49 s, total: 3min 15s
Wall time: 3min 15s


array([5., 0., 8., ..., 2., 2., 7.])

In [41]:
%time clf.predict_proba(X_test)

CPU times: user 7min 51s, sys: 12 ms, total: 7min 51s
Wall time: 7min 51s


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [42]:
%time my_clf.predict_proba(X_test)

CPU times: user 3min 14s, sys: 1.71 s, total: 3min 16s
Wall time: 3min 16s


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [43]:
clf.score(X_test, y_test)

0.9691428571428572

In [44]:
my_clf.score(X_test, y_test)

[5. 0. 8. ... 2. 2. 7.] [5. 0. 8. ... 2. 2. 7.]


0.9998571428571429

Задание 5

In [ ]:
# seaborn
# matplotlib
# train_test_split
# Pipelin%load_ext e (from sklearn.pipeline import Pipeline)
# StandardScaler (from sklearn.preprocessing import StandardScaler)
# ClassifierMixin
# NeighborsBase
# KNeighborsMixin
# SupervisedIntegerMixin